# As-Built Autopilot - Complete Workflow

**Author:** John Kenny, Municipal GIS Partners

This notebook demonstrates the complete As-Built Autopilot workflow:
1. Interactive georeferencing of raw as-built documents
2. AI-powered feature segmentation with SAM3
3. Vector conversion (points, lines, polygons)
4. Export to geodatabase/shapefile for use in ArcGIS Pro

## Installation

```bash
pip install "segment-geospatial[samgeo3]" leafmap rasterio geopandas ipywidgets
pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121
pip install transformers
```

## Step 1: Import Libraries and Initialize SAM3

In [ ]:
import sys
sys.path.append('/path/to/asbuilt_autopilot.py')  # Update this path

from samgeo import SamGeo3
from asbuilt_autopilot import AsBuiltAutopilot
import torch

# Check GPU availability
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'N/A'}")

## Step 2: Authenticate with Hugging Face (One Time)

SAM3 is a gated model. Request access at: https://huggingface.co/facebook/sam3

In [ ]:
from huggingface_hub import login
login("enter your token here")  # Enter your Hugging Face token

## Step 3: Initialize SAM3 Model

In [ ]:
# Initialize SAM3 with GPU support
sam3 = SamGeo3(
    backend="meta", #or transformers
    device="cuda",  # Use your RTX 5070 Ti!
    checkpoint_path=None,
    load_from_HF=True
)

print("✓ SAM3 initialized")

## Step 4: Initialize As-Built Autopilot

Set your target CRS (default is Illinois State Plane East NAD83 Feet - EPSG:3435)

In [ ]:
# Initialize autopilot with your target CRS
autopilot = AsBuiltAutopilot(
    sam3_model=sam3,
    target_crs="EPSG:3435"  # Change to your project CRS
)

print("✓ As-Built Autopilot ready")

## Step 5A: Interactive Georeferencing (For Raw As-Builts)

Use this if you have a non-georeferenced as-built (PNG, JPG, PDF)

In [ ]:
# Launch interactive georeferencing interface
autopilot.start_georeferencing(
    image_path="/path/to/raw_asbuilt.pdf",
    initial_center=[41.8781, -87.6298],  # Chicago area
    initial_zoom=16
)

# Instructions:
# 1. Click 'Add GCP' button
# 2. Click matching points on image and map
# 3. Repeat for at least 4 control points
# 4. Click 'Generate Georeferenced Image'

## Step 5B: Load Existing Georeferenced As-Built

Use this if you already have a georeferenced GeoTIFF

In [ ]:
# Load pre-georeferenced image
metadata = autopilot.load_georeferenced_asbuilt(
    image_path="/path/to/georeferenced_asbuilt.tif"
)

print(metadata)

## Step 6: Interactive Segmentation Interface

This is the main interface for:
- Segmenting features with text prompts
- Converting to vector geometries
- Exporting to geodatabase/shapefile

In [ ]:
# Launch interactive segmentation interface
autopilot.create_segmentation_interface(height="800px")

# Workflow in the UI:
# 1. Enter a text prompt (e.g., "valve", "manhole", "water pipe")
# 2. Click "Segment Features"
# 3. Select output geometry type (Points/Lines/Polygons)
# 4. Fill in attribute fields (optional)
# 5. Click "Convert to Vector"
# 6. Specify export path and format
# 7. Click "Export Features"

## Alternative: Programmatic Workflow

If you prefer code over UI, here's the programmatic approach:

In [ ]:
# Set image for segmentation
autopilot.set_image_for_segmentation()

# Segment features with text prompt
autopilot.segment_features(prompt="water valve")

# Convert to points
autopilot.masks_to_points(
    attribute_template={
        'feature_type': 'water_valve',
        'material': 'cast_iron',
        'size': '8"'
    }
)

# Reproject to target CRS
autopilot.reproject_features()

# Export to geodatabase
autopilot.export_to_geodatabase(
    gdb_path="/path/to/project.gdb",
    feature_class_name="water_valves"
)

print("✓ Features exported!")

## Example Workflows

### Extracting Water Valves as Points

In [ ]:
# Segment valves
autopilot.segment_features(prompt="water valve")

# Convert to points (centroids)
autopilot.masks_to_points(
    attribute_template={'feature_type': 'water_valve'}
)

# Export
autopilot.export_to_geodatabase(
    gdb_path="/path/to/utilities.gdb",
    feature_class_name="water_valves"
)

### Extracting Pipes as Lines

In [ ]:
# Segment pipes
autopilot.segment_features(prompt="pipe")

# Convert to lines (skeletonized)
autopilot.masks_to_lines(
    attribute_template={
        'feature_type': 'water_main',
        'material': 'ductile_iron',
        'diameter': '12"'
    },
    min_length=5.0  # Filter out short segments
)

# Export
autopilot.export_to_shapefile("/path/to/water_mains.shp")

### Extracting Manholes as Polygons

In [ ]:
# Segment manholes
autopilot.segment_features(prompt="manhole")

# Convert to polygons
autopilot.masks_to_polygons(
    attribute_template={'feature_type': 'storm_manhole'},
    simplify_tolerance=0.5
)

# Export
autopilot.export_to_geodatabase(
    gdb_path="/path/to/stormwater.gdb",
    feature_class_name="manholes"
)

## Batch Processing Multiple As-Builts

In [ ]:
from pathlib import Path

# Directory containing georeferenced as-builts
asbuilt_dir = Path("/path/to/asbuilts/")
output_gdb = Path("/path/to/output.gdb")

# Process each as-built
for asbuilt_file in asbuilt_dir.glob("*.tif"):
    print(f"\n{'='*60}")
    print(f"Processing: {asbuilt_file.name}")
    print(f"{'='*60}")
    
    # Load image
    autopilot.load_georeferenced_asbuilt(asbuilt_file)
    autopilot.set_image_for_segmentation()
    
    # Segment valves
    autopilot.segment_features(prompt="valve")
    autopilot.masks_to_points(
        attribute_template={
            'source_doc': asbuilt_file.stem
        }
    )
    
    # Export with unique name
    fc_name = f"valves_{asbuilt_file.stem}"
    autopilot.export_to_geodatabase(output_gdb, fc_name)
    
    print(f"✓ Completed: {fc_name}")

print("\n🎉 Batch processing complete!")

## Tips for Best Results

### Text Prompts
- Be specific: "water valve" vs "valve"
- Try variations: "manhole", "storm inlet", "catch basin"
- For pipes, try: "pipe", "water line", "sewer line"

### Georeferencing
- Use at least 4 GCPs, 6-8 is better
- Distribute GCPs across the entire document
- Choose clear, identifiable features (building corners, curb intersections)
- Verify accuracy by checking known distances

### Vector Conversion
- **Points**: Best for valves, manholes, hydrants
- **Lines**: Best for pipes, cables, conduits
- **Polygons**: Best for buildings, ponds, large structures

### Performance
- GPU acceleration is crucial for large documents
- Process at native resolution for best accuracy
- Consider tiling very large documents (>10,000 x 10,000 px)

## Integration with ArcGIS Pro

After exporting features:

1. Open ArcGIS Pro
2. Add the output geodatabase to your project
3. Features will be in your target CRS (EPSG:3435)
4. Use attribute fields for symbology and labeling
5. Perform quality control and manual edits as needed

### Python Toolbox Integration

You can wrap this workflow in an ArcGIS Python Toolbox for direct use in Pro!